## Step 1 web scraping with python

#### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

#### Using BeautifulSoup for scrape with HTTP Request and HTML Parser

In [2]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

#### Finding table body and create new data frame for postcode of Toronto

In [3]:
tableBody = soup.find_all('tbody', limit=1)[0]

dfPostalToronto = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighborhood'])

#### Parsing with rows of html table and add item to data frame

In [4]:
for tag in tableBody.find_all('tr'):
    postcod = tag.find_next()
    borough = postcod.find_next()
    neighborhood = borough.find_next()
    
    str_borough = borough.string     
    # if exists tag "a" get content tag "a" and go to next tag
    if borough.a is not None :
      str_borough =  borough.a.string.strip()
      neighborhood = neighborhood.find_next()
        
    str_neighborhood = neighborhood.string   
    # if exists tag "a" get content tag "a" and go to next tag
    if neighborhood.a is not None :
      str_neighborhood =  neighborhood.a.string.strip()
    
    dfPostalToronto = dfPostalToronto.append({
                        'Postcode'     : postcod.string.strip(), 
                        'Borough'      : str_borough, 
                        'Neighborhood': str_neighborhood.strip()}, ignore_index=True)    

#### Preparing data and exclude boroughs not assigned

In [5]:
#remove header of table
dfPostalToronto = dfPostalToronto.iloc[1:]

#exclude Boroughs Not assigned
dfPostalToronto = dfPostalToronto.loc[dfPostalToronto['Borough']  != 'Not assigned']

#change Neighbourhood Not assigned to Borough
nbAux = np.where(dfPostalToronto.Neighborhood.eq('Not assigned'), dfPostalToronto.Borough, dfPostalToronto.Neighborhood)
dfPostalToronto.Neighborhood = nbAux 

#### Grouping data by Postcode and Borough

In [6]:
dfPostalToronto = dfPostalToronto.groupby(['Postcode', 'Borough'])['Neighborhood'].agg([('Neighborhood', ', '.join)])
dfPostalToronto = dfPostalToronto.reset_index()

#### Display number of rows after agregate

In [7]:
dfPostalToronto.shape

(103, 3)

## Step 2 finding geo location

#### Importing CSV

In [8]:
pdGeoToronto = pd.read_csv('Geospatial_Coordinates.csv')

#### join dataframes by Postcode

In [9]:
dfPostalToronto = dfPostalToronto.merge(pdGeoToronto, left_on='Postcode', right_on='Postal Code')
#remove duplicated column
dfPostalToronto.drop('Postal Code', axis=1, inplace=True)

In [10]:
dfPostalToronto

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Step 3 Explore and cluster the neighborhoods in Toronto

#### Importing Libraries

In [11]:
from geopy.geocoders import Nominatim
import folium # map rendering library
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#### Getting location of East Toronto

In [12]:
address = 'East Toronto, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of East Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of East Toronto are 43.653963, -79.387207.


In [13]:
dfEastToronto = dfPostalToronto.loc[dfPostalToronto['Borough']  == 'East Toronto'].reset_index(drop=True)

#### Create map of East Toronto using latitude and longitude values

In [14]:
map_east_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfEastToronto['Latitude'], dfEastToronto['Longitude'], dfEastToronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_east_toronto)  
    
map_east_toronto

In [15]:
neighborhood_latitude = dfEastToronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfEastToronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfEastToronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = '4QDE4CQ4F5SGJB2T2D4HY5CTVI1AECBF5JANQZ2EGUDMLXV3' # your Foursquare ID
CLIENT_SECRET = 'MM1DJ0WYF1T35LT1R1RJTA340F1NKE2SU5UCPUF2DXVZIZFZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4QDE4CQ4F5SGJB2T2D4HY5CTVI1AECBF5JANQZ2EGUDMLXV3
CLIENT_SECRET:MM1DJ0WYF1T35LT1R1RJTA340F1NKE2SU5UCPUF2DXVZIZFZ


#### Now, let's get the top 100 venues that are in East Toronto within a radius of 500 meters

In [17]:
# type your answer here
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=4QDE4CQ4F5SGJB2T2D4HY5CTVI1AECBF5JANQZ2EGUDMLXV3&client_secret=MM1DJ0WYF1T35LT1R1RJTA340F1NKE2SU5UCPUF2DXVZIZFZ&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [18]:
results = requests.get(url).json()

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Starbucks,Coffee Shop,43.678798,-79.298045
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


In [21]:
# one hot encoding
eastToronto_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
eastToronto_onehot['Venues'] = nearby_venues['name']

# move neighborhood column to the first column
fixed_columns = [eastToronto_onehot.columns[-1]] + list(eastToronto_onehot.columns[:-1])
eastToronto_onehot = eastToronto_onehot[fixed_columns]

eastToronto_onehot.head()

,Venues,Coffee Shop,Health Food Store,Neighborhood,Pub
0,The Big Carrot Natural Food Market,0,1,0,0
1,Grover Pub and Grub,0,0,0,1
2,Starbucks,1,0,0,0
3,Upper Beaches,0,0,1,0
4,Dip 'n Sip,1,0,0,0


#### Explore Neighborhoods in East Toronto

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
eastToronto_venues = getNearbyVenues(names=dfEastToronto['Neighborhood'],
                                   latitudes=dfEastToronto['Latitude'],
                                   longitudes=dfEastToronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Business Reply Mail Processing Centre 969 Eastern


In [24]:
eastToronto_venues.shape

(124, 7)

In [25]:
# one hot encoding
eastToronto_onehot = pd.get_dummies(eastToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
eastToronto_onehot['Neighborhood'] = eastToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [eastToronto_onehot.columns[-1]] + list(eastToronto_onehot.columns[:-1])
eastToronto_onehot = eastToronto_onehot[fixed_columns]

eastToronto_onehot.head()

,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,...,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
eastToronto_grouped = eastToronto_onehot.groupby('Neighborhood').mean().reset_index()
eastToronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,...,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,0.00000,...,0.000000,0.000000,0.058824,0.058824,0.058824,0.00000,0.000000,0.000000,0.000000,0.00000
1,Studio District,0.025641,0.051282,0.000000,0.051282,0.025641,0.025641,0.025641,0.025641,0.00000,...,0.025641,0.025641,0.000000,0.000000,0.000000,0.00000,0.025641,0.000000,0.000000,0.00000
2,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
3,"The Beaches West, India Bazaar",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.00000,...,0.095238,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.047619,0.047619,0.00000
4,"The Danforth West, Riverdale",0.023810,0.023810,0.000000,0.023810,0.000000,0.000000,0.047619,0.023810,0.02381,...,0.000000,0.000000,0.000000,0.000000,0.023810,0.02381,0.000000,0.000000,0.000000,0.02381


In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = eastToronto_grouped['Neighborhood']

for ind in np.arange(eastToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(eastToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business Reply Mail Processing Centre 969 Eastern,Garden Center,Pizza Place,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden,Light Rail Station
1,Studio District,Café,Coffee Shop,American Restaurant,Bakery,Gastropub,Italian Restaurant,Fish Market,Coworking Space,Convenience Store,Comfort Food Restaurant
2,The Beaches,Coffee Shop,Pub,Health Food Store,Dessert Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Trail
3,"The Beaches West, India Bazaar",Park,Sandwich Place,Fast Food Restaurant,Pub,Gym,Italian Restaurant,Liquor Store,Burrito Place,Movie Theater,Burger Joint
4,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Dessert Shop,Indian Restaurant,Grocery Store,Furniture / Home Store,Fruit & Vegetable Store


In [29]:
eastToronto_grouped.drop('Neighborhood', 1).head()

,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,...,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,0.00000,0.000000,...,0.000000,0.000000,0.058824,0.058824,0.058824,0.00000,0.000000,0.000000,0.000000,0.00000
1,0.025641,0.051282,0.000000,0.051282,0.025641,0.025641,0.025641,0.025641,0.00000,0.000000,...,0.025641,0.025641,0.000000,0.000000,0.000000,0.00000,0.025641,0.000000,0.000000,0.00000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.00000,0.047619,...,0.095238,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.047619,0.047619,0.00000
4,0.023810,0.023810,0.000000,0.023810,0.000000,0.000000,0.047619,0.023810,0.02381,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.023810,0.02381,0.000000,0.000000,0.000000,0.02381


In [30]:
# set number of clusters
kclusters = 5

eastToronto_grouped_clustering = eastToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(eastToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 1, 2, 0], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

eastToronto_merged = dfEastToronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
eastToronto_merged = eastToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

eastToronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Coffee Shop,Pub,Health Food Store,Dessert Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Trail
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Dessert Shop,Indian Restaurant,Grocery Store,Furniture / Home Store,Fruit & Vegetable Store
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,2,Park,Sandwich Place,Fast Food Restaurant,Pub,Gym,Italian Restaurant,Liquor Store,Burrito Place,Movie Theater,Burger Joint
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Café,Coffee Shop,American Restaurant,Bakery,Gastropub,Italian Restaurant,Fish Market,Coworking Space,Convenience Store,Comfort Food Restaurant
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,3,Garden Center,Pizza Place,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden,Light Rail Station


In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(eastToronto_merged['Latitude'], eastToronto_merged['Longitude'], eastToronto_merged['Neighborhood'], eastToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters